In [54]:
import regex
import json
import numpy as np
import os
import pandas as pd
import re
import math
from typing import Any, Optional, Tuple

import os
import json
# from google.colab import drive

# Mount Google Drive
# drive.mount('/content/drive')

# Path to base folder in Drive
base_dir = '/content/drive/MyDrive/cot-analysis/cot_length'

def count_word_multilingual(text: str) -> int:
    """
    Performs rough tokenization of the input text and counts the number of tokens.
    Supports mixed languages including English, Chinese (including Japanese Kanji), 
    Spanish, Portuguese, French, German, Russian, Norwegian, and Japanese.
    
    Rules:
    1) [0-9\\.]+ : Matches consecutive digits (including decimal points) as a single token.
    2) \\p{Han} : Matches a single Chinese character (including Kanji used in Japanese).
    3) \\p{Hiragana}+ : Matches consecutive Hiragana characters as a single token.
    4) \\p{Katakana}+ : Matches consecutive Katakana characters as a single token.
    5) \\p{Cyrillic}+ : Matches consecutive Cyrillic letters (Russian).
    6) \\p{Latin}+ : Matches consecutive Latin letters (including diacritics),
                     supporting English, Spanish, Portuguese, French, German, Norwegian, etc.
    
    Notes:
    - Each Chinese character (\\p{Han}) is treated as an individual token. 
      For example, "你好" => ["你", "好"].
    - Consecutive characters from other scripts (e.g., "hello") are treated as a single token.
    - This is a simplified example and does not handle other symbols, punctuation, 
      or complex numerical formats.
    - Requires the third-party module `regex` (pip install regex),
      because the built-in `re` module has incomplete support for Unicode properties \\p{...}.
    """
    pattern = (
        r'[0-9\.]+'        # Consecutive digits and decimal points
        r'|\p{Han}'        # Single Chinese character
        r'|\p{Hiragana}+'  # Consecutive Hiragana characters
        r'|\p{Katakana}+'  # Consecutive Katakana characters
        r'|\p{Cyrillic}+'  # Consecutive Cyrillic letters
        r'|\p{Latin}+'     # Consecutive Latin letters (including diacritics)
    )
    tokens = regex.findall(pattern, text)
    return tokens, len(tokens)


def sort_files(files_list):

    num_re = re.compile(r"^(\d+)")  # capture 1+ digits at start

    def leading_num(fname):
        m = num_re.match(fname)
        if m:
            return int(m.group(1))
        else:
            return float("inf")  # or 0, depending on where you want no‑number files

    sorted_files = sorted(files_list, key=leading_num)
    
    return sorted_files

def split_by_analysis(
    instructions: str,
    prediction: str,
    record_id: Any
) -> Tuple[str, Optional[str], Any]:
    """
    Split a model’s output into (analysis, result) based on markers.

    Tries, in order:
    1. Splitting on the literal "Result:" in the prediction.
    2. Deriving the result label from instructions (e.g. "Result: XYZ:"),
       then splitting on "XYZ:" in the prediction.
    Falls back to returning the whole prediction as analysis with result=None.
    """
    # Should first split by <think>

    # 1) Try literal "Result:" in prediction
    try:
        analysis, result = prediction.split("Result:", 1)
        return analysis.strip(), result.strip(), record_id
    except ValueError:
        pass  # no "Result:" marker in prediction

    # 2) Derive the label from instructions, e.g. instructions contains "Result: XYZ:"
    match = re.search(r"Result:\s*([^:\s]+):", instructions)

    if match:
        label = match.group(1)
        marker = f"{label}:"
        try:
            analysis, result = prediction.split(marker, 1)
            return analysis.strip(), result.strip(), record_id
        except ValueError:
            pass

    # 3) Fallback
    return prediction.strip(), None, record_id


In [ ]:
# OUTPUT FORMAT
d = {
    "Task Name": '',
    "Model Name": '',
    "Number of Tokens": [],
}

In [55]:
d = {
        "task": "91-1.CAS.label",
        "language": "fr",
        "type": "ext",
        "id": 702,
        "split": "test",
        "instruction": "Given the clinical care report in French, extract the following medical information:\n- \"age\":  l'âge de la personne dont le cas est décrit, au moment du dernier élément clinique rapporté dans le cas clinique, normalisé sous la forme d'un entier (soit 0 pour un nourrisson de moins d'un an, 1 pour un enfant de moins de deux ans, y compris un an et demi, 20 pour un patient d'une vingtaine d'années, etc.).\n- \"genre\": le genre de la personne dont le cas est décrit, parmi deux valeurs normalisées : féminin, masculin (il n'existe aucun cas de dysgénésie ou d'hermaphrodisme dans le corpus). si le genre n'est pas mentionné, retournez \"None\".\n- \"issue\": l'issue parmi cinq valeurs possibles: (1) guérison (le problème clinique décrit dans le cas a été traité et la personne est guérie), (2) amélioration (l'état clinique est amélioré sans qu'on ne puisse conclure à une guérison), (3) stable (soit l'état clinique reste stationnaire, soit il est impossible de déterminer entre amélioration et détérioration), (4) détérioration (l'état clinique se dégrade), ou (5) décès (lorsque le décès concerne directement le cas clinique décrit). si le problème n'est pas mentionné, retournez \"None\".\nSolve it in a step-by-step fashion, return your answer in the following format, PROVIDE DETAILED ANALYSIS BEFORE THE RESULT:\nAnalysis:\n...\nResult:\nage: ..., genre: ..., issue: ...\nThe optional list for \"genre\" is [\"féminin\", \"masculin\", \"None\"].\nThe optional list for \"issue\" is [\"guérison\", \"amélioration\", \"stable\", \"détérioration\", \"décès\", \"None\"].",
        "input": "Mlle L… 25 ans a été adressée aux urgences du CHU de Bordeaux pour malaise le 26 septembre 2001. Elle présentait une hématurie avec caillots, évoluant depuis une semaine ainsi que des signes cliniques d’anémie.\n\nLes examens biologiques mettaient en évidence une anémie importante avec une hémoglobinémie à 5 grammes/100 ml sans autres anomalies à l’hémogramme, la fonction rénale était normale.\n\nHospitalisée en hématologie, une échographie ne mettait pas en évidence d’anomalies sur le haut appareil urinaire et trouvait dans la vessie des formations hyperéchogènes évoquant des caillots. Un sondage avec une sonde rigide confirmait la présence de caillots. Des manœuvres de décaillotage ont été entreprises au lit de la malade et une irrigation de sérum physiologique a été mise en place.\n\nLe 27 septembre 2001 a été réalisée au bloc opératoire une cystoscopie. Après décaillotage on constatait une muqueuse vésicale inflammatoire avec suffusions hémorragiques diffuses de toute la paroi vésicale sans lésion hémorragique particulière. Une nouvelle irrigation a été mise en place et un traitement hémostatique par acide tranexamique (EXACYL®) a été entrepris.\n\nLe 2 octobre 2001, un scanner abdomino-pelvien avec injection retrouvait un épaississement irrégulier de la paroi vésicale avec un contenu hétérogène de la vessie sans anomalies par ailleurs. Le haut appareil était normal et non dilaté. Le 3 octobre 2001 une nouvelle cystoscopie avec un décaillotage et des biopsies de la paroi vésicale a été réalisée. Une irrigation continue avec des sels d’aluminium a été commencée. Les urines sont devenues claires au troisième jour. L’examen anatomopathologique mettait en évidence une hyperplasie urothéliale avec discrètes atypies de nature indéterminée sans foyer tumoral visible. Les examens bactériologiques urinaires ont été négatifs. L’évolution étant satisfaisante, la sortie de la patiente a été décidée.\n\nLe 19 octobre 2001, Mme L... a été de nouveau hospitalisée pour hématurie avec caillots, les examens biologiques retrouvaient une anémie avec une hémoglobinémie à 5,4 g/100 ml. Le bilan étiologique par des analyses virologiques (recherche de cytomégalovirus, BK virus et JC virus dans le sang et les urines) et une cytologie urinaire ont été réalisés.\n\nUne nouvelle irrigation par sels d’aluminium a été débutée et continuée pendant 8 jours. Deux jours plus tard on a constaté un caillotage de la vessie et l’hémoglobine à 5,9 g/100 ml malgré les transfusions. On a réalisé une cystoscopie et 2 sondes urétérales ont été mises en place.\n\nLe bilan biologique retrouvait une PCR positive à JC virus dans le sang et à BK virus dans les urines. La cytologie urinaire ne retrouvait pas de signes d’infestation cellulaire par un agent viral. Le 30/10/01 a été entrepris un traitement antiviral par cidofoviristide®. Les sondes urétérales et vésicales ont été enlevées le 04/11/01, l’irrigation par sels d’aluminium a été arrêtée. Le 07/11/01 l’hématurie a été de nouveau importante avec caillots et déglobulisation et une hémoglobulinémie a 6,5g/100 ml.\n\nOn a réalisé de nouveau une cystoscopie et des sondes urétérales ont été mises en place. Le traitement antiviral a été continué. Un traitement par instillation endovésicale de prostaglandines a été débuté et poursuivi pendant 7 jours. La sonde vésicale a été enlevée ; il existait encore un caillotage vésical. Une semaine plus tard après une cystoscopie et un nouveau décaillotage, les urines sont restées claires et les sondes urétérales et vésicales ont été enlevées 3 jours plus tard. L’hémoglobine était stable a 8,8g/100 ml. Au total 24 culots globulaires auront été transfusés. La patiente a été autorisée à sortir le 21/11/01.\n\nRevue à la consultation 3 semaines plus tard, l’hématurie s’était arrêtée, l’hémoglobine était à 8,5g/100 ml. Les prélèvements viraux de contrôle ne retrouvaient pas de virus. Trois mois plus tard un nouveau contrôle urinaire ne retrouvait pas de trace du virus, l’hémoglobine était à 10,5 g/100 ml. La patiente se plaignait d’une pollakiurie, un examen urodynamique retrouvait une vessie hypocompliante avec une augmentation des pressions probablement secondaire aux traitements endovésicaux.",
        "output": "age: 25, genre: féminin, issue: stable",
        "pred": "Analyse :\n\nLe texte décrit le cas d'une patiente, Mlle L..., âgée de 25 ans, qui a été hospitalisée pour malaise et hématurie avec caillots en septembre 2001. Les examens biologiques ont révélé une anémie importante et une fonction rénale normale. Les traitements ont inclus des irrigations, des cystoscopies, des biopsies et des traitements antiviraux. La patiente a eu plusieurs épisodes de récidive, mais a finalement été guérie après un traitement prolongé.\n\nRésultat :\n\nage : 20\ngenre : féminin\nissue : guérison",
        "CoT Length": 83
    }


pred, _, _ = split_by_analysis(
    d['instruction'],
    d['pred'],
    d['id']
)

print(pred)
count_word_multilingual(s)

Analyse :

Le texte décrit le cas d'une patiente, Mlle L..., âgée de 25 ans, qui a été hospitalisée pour malaise et hématurie avec caillots en septembre 2001. Les examens biologiques ont révélé une anémie importante et une fonction rénale normale. Les traitements ont inclus des irrigations, des cystoscopies, des biopsies et des traitements antiviraux. La patiente a eu plusieurs épisodes de récidive, mais a finalement été guérie après un traitement prolongé.

Résultat :

age : 20
genre : féminin
issue : guérison


(['Analyse',
  'Le',
  'texte',
  'décrit',
  'le',
  'cas',
  'd',
  'une',
  'patiente',
  'Mlle',
  'L',
  '...',
  'âgée',
  'de',
  '25',
  'ans',
  'qui',
  'a',
  'été',
  'hospitalisée',
  'pour',
  'malaise',
  'et',
  'hématurie',
  'avec',
  'caillots',
  'en',
  'septembre',
  '2001.',
  'Les',
  'examens',
  'biologiques',
  'ont',
  'révélé',
  'une',
  'anémie',
  'importante',
  'et',
  'une',
  'fonction',
  'rénale',
  'normale',
  '.',
  'Les',
  'traitements',
  'ont',
  'inclus',
  'des',
  'irrigations',
  'des',
  'cystoscopies',
  'des',
  'biopsies',
  'et',
  'des',
  'traitements',
  'antiviraux',
  '.',
  'La',
  'patiente',
  'a',
  'eu',
  'plusieurs',
  'épisodes',
  'de',
  'récidive',
  'mais',
  'a',
  'finalement',
  'été',
  'guérie',
  'après',
  'un',
  'traitement',
  'prolongé',
  '.',
  'Résultat',
  'age',
  '20',
  'genre',
  'féminin',
  'issue',
  'guérison'],
 83)

In [ ]:
sheet = pd.ExcelFile("/Users/kevinxie/Desktop/LLM CoT/LLM-CoT/Clinical Benchmark and LLM.xlsx")
data = sheet.parse("Task-all")

destination = "/Users/kevinxie/Library/CloudStorage/GoogleDrive-kevinxie2024@gmail.com/My Drive/cot-analysis/combined-results"

task_folders = os.listdir(destination)

valid_tasks = []
for task_name in data['Task-Original']:
    valid_tasks.append(task_name)

### Outline

- There is an output file (.json) for each model-task pair. 
- We create 10 folders, each representing the 10% increments in length
- For each output file, we get the lengths of EACH individual output
- Then sort them
- Bucket and add to each of the 10 folders



## Simplified Task Name Mapping

In [ ]:
# Define task name to simplified task name mapping
google_sheet = pd.ExcelFile("Clinical Benchmark and LLM.xlsx")

data = google_sheet.parse("Task-all")

task_name_mapping = {}

for row_idx, task_name in enumerate(data["Task-Original"]):
    task_name_mapping[data["Task name"][row_idx]] = data["Task name"][row_idx]
    task_name_mapping[task_name] = data["Task name"][row_idx]


In [ ]:
def aug_cot_length(json_path):
    """
    Given a json file of data, return the number of tokens in the analysis and result
    and add to each entry.

    RETURNS:
        - Augments the json file with the length of CoT analysis
        - returns the same list of dictionaries
    """

    with open(json_path, 'r') as f:
        json_content = json.load(f)
    
    for d in json_content:
        analysis, _, _ = split_by_analysis(d["instruction"], d["pred"], d["id"])
        _, num_tokens = count_word_multilingual(analysis)

        d['CoT Length'] = num_tokens

    return json_content

def bucket(json_content):
    """
    Given a json file of data with each output's CoT length,
    bucket them into 10 different buckets (0-10, 10-20, ..., 90-100)
    and return as 10 different files

    RETURNS:
        - returns a list of 10 lists, each containing the data for that bucket, in decreasing order
    """
    # Step 1: Sort the data in descending order of "CoT Length"
    sorted_data = sorted(json_content, key=lambda x: x['CoT Length'], reverse=True)

    # Step 2: Split into 10 buckets (deciles)
    total_items = len(sorted_data)
    bucket_size = math.ceil(total_items / 10)

    decile_buckets = [
        sorted_data[i * bucket_size: (i + 1) * bucket_size]
        for i in range(10)
    ]

    return decile_buckets

# Function to convert decile index into folder name
def decile_folder(i):
    return 't_10' if i == 0 else f't_{i*10}_{(i+1)*10}'



# json_content = get_cot_length("/Users/kevinxie/Library/CloudStorage/GoogleDrive-kevinxie2024@gmail.com/My Drive/cot-analysis/combined-results/68.NUBES/Mistral-Small-3.1-24B-Instruct-2503/68.NUBES-cot-greedy-42.result.json")

# bucket(json_content)

In [ ]:
for task_name in task_folders:

    task_path = os.path.join(destination, task_name)
    if not os.path.isdir(task_path):
        continue

    for model_name in os.listdir(task_path):

        model_path = os.path.join(task_path, model_name)
        if not os.path.isdir(model_path):
            continue

        # gather *all* the CoT files for this (task, model) pair
        cot_files = [f for f in os.listdir(model_path) if "-cot-" in f]
        if not cot_files:
            continue

        cot_file = cot_files[0]


        cot_file_path = os.path.join(model_path, cot_file)

        # Read the contents of the CoT file

        aug_data = aug_cot_length(cot_file_path)

        decile_buckets = bucket(aug_data)

        # Replace this with your actual decile_buckets
        # decile_buckets = [...]

        for i, bucket in enumerate(decile_buckets):
            if not bucket:
                continue  # Skip empty buckets

            # Create path like cot_length/t_10/task_name/model_name/
            decile_path = os.path.join(base_dir, decile_folder(i), task_name, model_name)

            # Create directories
            os.makedirs(decile_path, exist_ok=True)

            # Save JSON file
            file_path = os.path.join(decile_path, 'bucket.json')
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(bucket, f, ensure_ascii=False, indent=4)

            print(f'Saved decile bucket {i+1} to {file_path}')

        break
        
    break
        # For each output, update it to have the cot length attribute






        


10
